## 任务1：报名比赛，下载比赛数据集并完成读取

- 步骤1 ：登录&报名比赛：https://aistudio.baidu.com/aistudio/competition/detail/45/0/task-definition
- 步骤2 ：下载比赛数据集
- 步骤3 ：使用Pandas完成数据读取。

## 任务2：对句子对提取TFIDF以及统计特征，训练和预测

参考代码：https://www.kaggle.com/anokas/data-analysis-xgboost-starter-0-35460-lb
- 步骤1 ：对句子对（句子A和句子B统计）如下特征：
  - 句子A包含的字符个数、句子B包含的字符个数
  - 句子A与句子B的编辑距离
  - 句子A与句子B共有单词的个数
  - 句子A与句子B共有字符的个数
  - 句子A与句子B共有单词的个数 / 句子A字符个数
  - 句子A与句子B共有单词的个数 / 句子B字符个数
- 步骤2 ：计算TFIDF，并对句子A和句子B进行特征转换
- 步骤3 ：计算句子A与句子B的TFIDF向量的内积距离
- 步骤4 ：将上述特征送入分类模型，训练并预测，将结果预测提交到比赛网站。

In [5]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import jieba
import distance 
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
import xgboost as xgb
import re
from sklearn.metrics import roc_auc_score, auc, roc_curve, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, QuantileTransformer, KBinsDiscretizer, LabelEncoder, MinMaxScaler, PowerTransformer

In [7]:
## jieba分词 
# cut_all=True，全模式，“我来到北京清华大学”-->“ 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学”
def jieba_cut(sentence,method=True):
    word_list = jieba.lcut(sentence,cut_all=method) 
    return word_list 

   
# 分词后，两句子相同词占所有词（去重）的比例
def percent(q1_cut,q2_cut):
    inter_num = len(set(q1_cut) & set(q2_cut))
    percent = inter_num/len(set(q1_cut))
    return percent

## 获取停词 
# https://github.com/goto456/stopwords
def stopwords():
    stop_words =[]
    with open('./cn_stopwords.txt','r',encoding='UTF-8') as f:
        for i in f.readlines():
            i = i.replace('\n','')
            stop_words.append(i)
    return stop_words

# 词共享 比例
def word_match_share(row,stops):
    q1words = {}
    q2words = {}
    # 剔除停词
    for word in str(row['q1_cut']):
        if word not in stops:
            q1words[word] = 1
    for word in str(row['q2_cut']):
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

# 准备语料
def all_words(train):
    corpus = []
    # 遍历每行，q1分词，q2分词，合并
    for row_id in range(len(train)):
        row = train.iloc[row_id]
        all_words = list()
        all_words.extend([word for word in row['q1_cut'] if word not in stopwords()])
        all_words.extend([word for word in row['q2_cut'] if word not in stopwords()])
        corpus.append(' '.join(all_words))
    return corpus

# 定义权重
# 词个数为1的，权重为0，大于1的，权重为 1/(count+10000)
def get_weight(cnt, eps=10000, min_count=2):
    if cnt < min_count:
        return 0
    else:
        return 1 / (cnt + eps)


def tfidf_word_match_share(row,weight):
    q1words = {word:1 for word in row['q1_cut'] if word not in stopwords()}
    q2words = {word:1 for word in row['q2_cut'] if word not in stopwords()}
    if len(q1words)==0 or len(q2words)==0:
        return 0 
    
    # 获取共享词的权重
    shared_weights = [weight.get(w,0) for w in q1words.keys() if w in q2words] + [weight.get(w,0) for w in q2words.keys() if w in q1words]
    # 总权重
    total_weights = [weight.get(w, 0) for w in q1words] + [weight.get(w, 0) for w in q2words]
    # 共享词权重比例
    R = np.sum(shared_weights) / np.sum(total_weights)
    return R

In [ ]:
# 特征处理

def handle_feature(train):
    # 字符个数
    train['q1_len'] = train['q1'].apply(len)
    train['q2_len'] = train['q2'].apply(len)  

    train['q1_cut'] = train['q1'].apply(lambda x:jieba_cut(x)) 
    train['q2_cut'] = train['q2'].apply(lambda x:jieba_cut(x)) 
    # 分词后的词个数
    train['q1_cut_len'] = train['q1_cut'].apply(len)
    train['q2_cut_len'] = train['q2_cut'].apply(len)  

    # 编辑距离 
    # Levenshtein Distance 被称为编辑距离（Edit Distance），一个度量两个字符序列之间差异的字符串度量标准
    train['Lev_distance'] = train.apply(lambda x:distance.levenshtein(x['q1'],x['q2']),axis=1)
    train['q1_cut_percent'] = train.apply(lambda x: percent(x['q1_cut'],x['q2_cut']),axis=1)
    train['q2_cut_percent'] = train.apply(lambda x: percent(x['q2_cut'],x['q1_cut']),axis=1)

    train['word_match'] = train.apply(lambda x: word_match_share(x,stopwords()),axis=1)

    corpus = all_words(train)
    corpus = (' '.join(corpus).split())
    word_cnt = Counter(corpus)

    # 生成词权重，词--权重
    weight =  { word:get_weight(cnt) for word ,cnt in word_cnt.items()}
    train['tfidf_word_match'] = train.apply(lambda x:tfidf_word_match_share(x,weight),axis=1)
    return train



In [ ]:
# LigthGMB    
          
def train_lgb_kfold(X_train, y_train, X_test, n_fold=5):
    '''train lightgbm with k-fold split'''
    gbms = []
    kfold = StratifiedKFold(n_splits=n_fold, random_state=2021, shuffle=True)
    oof_preds = np.zeros((X_train.shape[0],))
    test_preds = np.zeros((X_test.shape[0],))

    for fold, (train_index, val_index) in enumerate(kfold.split(X_train, y_train)):
        X_tr, X_val, y_tr, y_val = X_train.iloc[train_index], X_train.iloc[val_index], y_train[train_index], y_train[val_index]
        dtrain = lgb.Dataset(X_tr, y_tr)
        dvalid = lgb.Dataset(X_val, y_val, reference=dtrain)

        params = {
            'objective': 'binary',
            'metric': 'auc',
            'num_leaves': 300,
            'boosting_type':'gbdt',
            'subsample_freq':1,
            'reg_alpha':0.5,
            'reg_lambda':0.5,
            'n_estimators':2000,
            'learning_rate': 0.05,
            'min_data_in_leaf': 150,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'n_jobs': -1,
            'seed': 2022,
            'verbose':-1
        }

        gbm = lgb.train(params,
                        dtrain,
                        num_boost_round=100,
                        valid_sets=[dtrain, dvalid],
                        verbose_eval=50,
                        early_stopping_rounds=20)

        oof_preds[val_index] = gbm.predict(X_val, num_iteration=gbm.best_iteration)
        test_preds += gbm.predict(X_test, num_iteration=gbm.best_iteration) / kfold.n_splits
        gbms.append(gbm)

    return gbms, oof_preds, test_preds


def train_lgb(train, test, feat_cols, label_col, n_fold=10):
    '''训练lightgbm'''
    X_train = train[feat_cols]
    y_train = train[label_col]
    X_test = test[feat_cols]
    gbms_lgb, oof_preds_lgb, test_preds_lgb = train_lgb_kfold(X_train, y_train, X_test, n_fold=n_fold)
    
    return gbms_lgb, oof_preds_lgb, test_preds_lgb


def model(data):
    train = pd.read_csv(data_dir+'/'+data+'/train.tsv',sep='\t',error_bad_lines=False,names=['q1','q2','label']).dropna()
    test = pd.read_csv(data_dir+'/'+data+'/test.tsv',sep='\t',error_bad_lines=False,names=['q1','q2']).dropna()
    test['label'] = -1 
    dev = pd.read_csv(data_dir+'/'+data+'/dev.tsv',sep='\t',error_bad_lines=False,names=['q1','q2','label']).dropna()
    print(f'开始处理{data}数据')
    
    # label非[0,1]的不处理
    if len(set(train.label))>2:
        train = train[train['label'].isin(['0', '1'])]
        train['label'] = train['label'].astype('int')
    print('数据读取完成~')
    
    print('特征处理~')
    train = handle_feature(train)
    test = handle_feature(test)
    print('特征处理完成~')
    
    feat_cols = ['Lev_distance','word_match', 'tfidf_word_match']
    
    # 因为有drop，注意训练集的index的连续性
    train.index = train.reset_index(drop=True).index
    
    print('模型训练中~')
    gbms_lgb, oof_preds_lgb, test_preds_lgb = train_lgb(train, test,
                                                    feat_cols=feat_cols,
                                                    label_col='label')
    
    
    # 测试集预测结果
    df_test_submit = pd.DataFrame({'index': test.index.to_list(),
                               'prediction': test_preds_lgb})
    ## 直接按 0.5 划分
    df_test_submit['prediction'] =  np.where(df_test_submit['prediction']>0.5,1,0)
    df_test_submit.to_csv('./submit_result/' + data + '.tsv', index=False, sep='\t')
    
    print('预测输出成功~')

In [31]:
data_dir = 'D:/study_hard/statistic/千言数据集'
data_list = ['bq_corpus','lcqmc','paws-x-zh']
# data_dir = 'E:/学习/千言数据集/'

for data in data_list:
    model(data)

开始处理bq_corpus数据
数据读取完成~
特征处理~
特征处理完成~
模型训练中~
[LightGBM] [Info] Number of positive: 38832, number of negative: 38742
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000185 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 503
[LightGBM] [Info] Number of data points in the train set: 77574, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500580 -> initscore=0.002320
[LightGBM] [Info] Start training from score 0.002320
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

## 任务3：加载中文词向量，自己训练中文词向量

- 步骤1 ：使用jieba对中文句子进行分词
- 步骤2 ：使用gensim中Word2Vec训练分词后的句子，得到词向量。

参考：https://zhuanlan.zhihu.com/p/114538417#/

https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py

![20220124224400](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220124224400.png)

Word2Vec是轻量级的神经网络，其模型仅仅包括输入层、隐藏层和输出层，模型框架根据输入输出的不同，主要包括CBOW和Skip-gram模型。 

- CBOW（Continuous-bag-of-words）的方式是在知道词的上下文的情况下预测当前词；

- Skip-gram是在知道了词的情况下,对词的上下文进行预测；

Word2Vec相对于其他词袋模型来说，解决了2个问题：
1. 词序信息，n-gram捕获局部词序，但会受到高维、稀疏矩阵影响；
2. 潜在单词的含义；
   
但是，无法推断不熟悉的词的向量，FastText可解决这个问题。


In [1]:
data_dir = 'D:/study_hard/statistic/千言数据集'
# data_dir = 'E:/学习/千言数据集/'

train = pd.read_csv(data_dir+'/'+'bq_corpus'+'/train.tsv',sep='\t',error_bad_lines=False,names=['q1','q2','label']).dropna()
train.index = train.reset_index(drop=True).index
test = pd.read_csv(data_dir+'/'+'bq_corpus'+'/test.tsv',sep='\t',error_bad_lines=False,names=['q1','q2']).dropna()
test['label'] = -1 
dev = pd.read_csv(data_dir+'/'+'bq_corpus'+'/dev.tsv',sep='\t',error_bad_lines=False,names=['q1','q2','label']).dropna()

Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [2]:
train['q1'].head()

0               用微信都6年，微信没有微粒贷功能
1                         微信消费算吗
2           交易密码忘记了找回密码绑定的手机卡也掉了
3    你好我昨天晚上申请的没有打电话给我今天之内一定会打吗？
4                        “微粒贷开通"
Name: q1, dtype: object

In [8]:
# 分词
train['q1_cut'] = train['q1'].apply(lambda x:jieba_cut(x,method=False)) 
train['q2_cut'] = train['q2'].apply(lambda x:jieba_cut(x,method=False)) 

# 剔除标点
remove_char_list = list('[·’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+')
def remove_chars(row):
    return [word for word in row if word not in remove_char_list]

train['q1_cut'] = train['q1_cut'].apply(lambda x:remove_chars(x)) 
train['q2_cut'] = train['q2_cut'].apply(lambda x:remove_chars(x)) 

# 删除停词
# def remove_stop_word(row):
#     l = [word for word in row if word not in stopwords()]
#     return l

# train['q1_cut'] = train['q1_cut'].map(lambda row:remove_stop_word(row))
# train['q2_cut'] = train['q2_cut'].map(lambda row:remove_stop_word(row))

Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\CEALLA~1\AppData\Local\Temp\jieba.cache
Loading model from cache C:\Users\CEALLA~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.532 seconds.
Loading model cost 0.532 seconds.
Prefix dict has been built successfully.
Prefix dict has been built successfully.


In [9]:
train['q1_cut'].head()

0                  [用, 微信, 都, 6, 年, 微信, 没有, 微粒, 贷, 功能]
1                                       [微信, 消费, 算, 吗]
2         [交易, 密码, 忘记, 了, 找回, 密码, 绑定, 的, 手机卡, 也, 掉, 了]
3    [你好, 我, 昨天晚上, 申请, 的, 没有, 打电话, 给, 我, 今天, 之内, 一定...
4                                          [微粒, 贷, 开通]
Name: q1_cut, dtype: object

In [12]:
train.head()

,q1,q2,label,q1_cut,q2_cut
0,用微信都6年，微信没有微粒贷功能,4。号码来微粒贷,0,"[用, 微信, 都, 6, 年, 微信, 没有, 微粒, 贷, 功能]","[4, 号码, 来, 微粒, 贷]"
1,微信消费算吗,还有多少钱没还,0,"[微信, 消费, 算, 吗]","[还有, 多少, 钱, 没, 还]"
2,交易密码忘记了找回密码绑定的手机卡也掉了,怎么最近安全老是要改密码呢好麻烦,0,"[交易, 密码, 忘记, 了, 找回, 密码, 绑定, 的, 手机卡, 也, 掉, 了]","[怎么, 最近, 安全, 老是, 要, 改, 密码, 呢, 好, 麻烦]"
3,你好我昨天晚上申请的没有打电话给我今天之内一定会打吗？,什么时候可以到账,0,"[你好, 我, 昨天晚上, 申请, 的, 没有, 打电话, 给, 我, 今天, 之内, 一定, 会, 打, 吗]","[什么, 时候, 可以, 到, 账]"
4,"“微粒贷开通""",你好，我的微粒贷怎么没有开通呢,0,"[微粒, 贷, 开通]","[你好, 我, 的, 微粒, 贷, 怎么, 没有, 开通, 呢]"


In [34]:
# 准备语料 
# 一句话一个list

def prepare_words(train):
    corpus = []
    # 遍历每行，q1分词，q2分词，合并
    for row_id in range(len(train)):
        row = train.iloc[row_id]
        all_words = []
        all_words.append([word for word in row['q1_cut']])
        all_words.append([word for word in row['q2_cut']])
        corpus.extend(all_words)
    return corpus


sentences = prepare_words(train)
sentences[:5]

[['用', '微信', '都', '6', '年', '微信', '没有', '微粒', '贷', '功能'],
 ['4', '号码', '来', '微粒', '贷'],
 ['微信', '消费', '算', '吗'],
 ['还有', '多少', '钱', '没', '还'],
 ['交易', '密码', '忘记', '了', '找回', '密码', '绑定', '的', '手机卡', '也', '掉', '了']]

In [35]:
# 使用gensim训练词向量
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

def w2v(data):
    n_dim = 100
    # 初始化模型,sg=1:skip-gram
    w2v_model = Word2Vec(sentences=data,size=n_dim,window=5,min_count=5,sg=1,workers=4,iter=5) 
    # 保存模型
    w2v_model.save('./w2v_model.pkl')
    # print(w2v_model.wv['信'])
    # 仅保存词+embeddings
    word_vec = w2v_model.wv
    word_vec.save("word2vec.wordvectors")


w2v_model = w2v(sentences)

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 65540 words, keeping 3195 word types
PROGRESS: at sentence #20000, processed 174643 words, keeping 4369 word types
PROGRESS: at sentence #30000, processed 244552 words, keeping 4817 word types
PROGRESS: at sentence #40000, processed 313983 words, keeping 5116 word types
PROGRESS: at sentence #50000, processed 413072 words, keeping 5358 word types
PROGRESS: at sentence #60000, processed 489678 words, keeping 5489 word types
PROGRESS: at sentence #70000, processed 554928 words, keeping 5565 word types
PROGRESS: at sentence #80000, processed 620694 words, keeping 5617 word types
PROGRESS: at sentence #90000, processed 685771 words, keeping 5680 word types
PROGRESS: at sentence #100000, processed 751360 words, keeping 5713 word types
PROGRESS: at sentence #110000, processed 833694 words, keeping 5745 word types
PROGRESS: at sentence #120000, proces

In [44]:
# 加载模型
# w2v_model.load("word2vec.model")
# w2v_model = Word2Vec.load('./w2v_model.pkl')

wv = w2v_model.wv # 得到训练之后的词向量
# del w2v_model # 删除占用大内存的model
wv.save('word_vector') # 保存word vectors
loaded_wv = KeyedVectors.load('word_vector', mmap='r') # 加载保存的word vectors
loaded_wv.wv['信']
# w2v_model.wv.similarity('微信','密码')
# w2v_model.wv.most_similar('微信')

saving Word2VecKeyedVectors object under word_vector, separately None
not storing attribute vectors_norm
saved word_vector
loading Word2VecKeyedVectors object from word_vector
setting ignored attribute vectors_norm to None
loaded word_vector


array([-0.06417012, -0.43138066,  0.24497293, -0.6318257 ,  0.46362594,
        0.16690534, -0.04542749, -0.06943604,  0.7452645 , -0.25690654,
        0.05823705,  0.17919533, -0.19950837, -0.0835789 ,  0.25919223,
        0.38819626, -0.23068449, -0.26988578, -0.1471273 ,  0.10673296,
       -0.29348758, -0.07418719, -0.05724544, -0.06824961,  0.04133222,
        0.08989551, -0.29969117, -0.04653722, -0.0051935 ,  0.1393903 ,
       -0.06400751,  0.1379385 , -0.24359508,  0.14472738, -0.39280763,
        0.34356555, -0.37925234, -0.16961291, -0.12785459,  0.4583149 ,
        0.37192208, -0.09565699, -0.11805826,  0.49284348,  0.05121926,
        0.2855874 , -0.02007485,  0.03887828,  0.52951187, -0.17341092,
        0.22164704,  0.20213088,  0.17750889, -0.15410167,  0.16346893,
       -0.12732734, -0.10066055, -0.22101301,  0.07070814, -0.28421578,
        0.07123972,  0.15696977, -0.14252259, -0.06257319,  0.10973185,
        0.15907256, -0.61640406, -0.04865155, -0.16424526, -0.14

## 任务4：使用中文词向量完成mean/max/sif句子编码

- 步骤1 ：单词通过word2vec编码为100维向量，则句子编码为N∗100的矩阵，N为句子单词个数。
- 步骤2 ：将N*100的矩阵进行max-pooling编码，转为100维度。
- 步骤3 ：将N*100的矩阵进行mean-pooling编码，转为100维度。
- 步骤4 ：将N*100的矩阵与单词的IDF进行矩阵相乘，即按照单词的词频进行加权，进行tfidf-pooling编码，转为100维度。
- 步骤5 ：学习SIF编码的原理，进行sif编码，转为100维度。
  - https://github.com/PrincetonML/SIF/blob/master/src/SIF_embedding.py#L30
  - https://openreview.net/pdf?id=SyK00v5xx
- 步骤6（可选） ：通过上述步骤2-步骤5的编码，计算相似句子的相似度 vs 不相似句子的相似度， 绘制得到分布图，哪一种编码最优？

参考：
https://blog.csdn.net/asialee_bird/article/details/100124565

**平均词向量**：

将句子中所有词的word embedding相加取平均，得到的向量当做最终的sentence embedding。该方法缺点是认为句子中的所有词对于表达句子含义同样重要。

![20220126003459](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220126003459.png)

**TFIDF加权平均词向量**：

对每个词按照tf-idf进行打分，然后进行加权平均，得到最终的句子表示。

![20220126003755](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220126003755.png)

mean-pooling

In [45]:
# mean-pooling: 对每个句子的所有词向量取均值，生成句子的vector

def build_sentence_mean_vector(sentences,w2v_model):
    arr = []
    for word in sentences:
        try:
            arr.append(w2v_model.wv[word])
        except KeyError:
            continue
    if len(arr) == 0:
        return np.zeros(shape=(1,100)).mean(0)
    else:
        return np.vstack(arr).mean(0) 


train['q1_sen_mean_vec'] = train['q1_cut'].map(lambda sentence:build_sentence_mean_vector(sentence,w2v_model))
train['q2_sen_mean_vec'] = train['q2_cut'].map(lambda sentence:build_sentence_mean_vector(sentence,w2v_model))

max-pooling

In [46]:
def build_sentence_max_vector(sentences,w2v_model):
    arr = []
    for word in sentences:
        try:
            arr.append(w2v_model.wv[word])
        except KeyError:
            continue
    if len(arr) == 0:
        return np.zeros(shape=(1,100)).max(0)
    else:
        return np.vstack(arr).max(0) 


train['q1_sen_max_vec'] = train['q1_cut'].map(lambda sentence:build_sentence_max_vector(sentence,w2v_model))
train['q2_sen_max_vec'] = train['q2_cut'].map(lambda sentence:build_sentence_max_vector(sentence,w2v_model))

['用 微信 都 6 年 微信 没有 微粒 贷 功能',
 '4 号码 来 微粒 贷',
 '微信 消费 算 吗',
 '还有 多少 钱 没 还',
 '交易 密码 忘记 了 找回 密码 绑定 的 手机卡 也 掉 了',
 '怎么 最近 安全 老是 要 改 密码 呢 好 麻烦',
 '你好 我 昨天晚上 申请 的 没有 打电话 给 我 今天 之内 一定 会 打 吗',
 '什么 时候 可以 到 账',
 '微粒 贷 开通',
 '你好 我 的 微粒 贷 怎么 没有 开通 呢',
 '为什么 借款 后 一直 没有 给 我 回 拨电话',
 '怎么 申请 借款 后 没有 打电话 过来 呢',
 '为什么 我 每次 都 提前 还款 了 最后 却 不 给 我 贷款 了',
 '30 号 我 一次性 还清 可以 不',
 '请问 一天 是否 都 是 限定 只能 转入 或 转出 都 是 五万',
 '微众 多少 可以 赎回 短期 理财',
 '微粒 咨询 电话号码 多少',
 '你们 的 人工 客服 电话 是 多少',
 '已经 在 银行 换 了 新 预留 号码',
 '我 现在 换 了 电话号码 这个 需要 更换 吗',
 '下周 有 什么 好 产品',
 '元月份 有 哪些 理财产品',
 '第一次 使用 额度 多少',
 '我 的 额度 多少 钱',
 '我 什么 时候 可以 通过 微粒 贷 借钱',
 '提前 还清 贷款 还 能 再 借 吗',
 '借款 后 多长时间 给 打电话',
 '借款 后 多久 打电话 啊',
 '没 看到 微粒 贷',
 '我 借 那么 久 也 没有 提升 啊',
 '原来 的 手机号 不用 了 怎么 换',
 '手机号码 没有 更改',
 '我 想 开通 微粒 贷 不知 我 应该 做 写 什么 准备 材料 呢',
 '为何 苹果 手机 显示 微粒 贷暂 未 开放',
 '能 查帐 单 吗',
 '可以 查询 帐单',
 '日利率 多少',
 '息 多少',
 '微信 6.2 的 版本 没有 微粒 贷 吗',
 '什么 时候 才 会 全面 开放 名额',
 '0K 谢谢 还 没有 开通',
 '开通 为什么 显示 证件 已 被 使用',
 '综合 评估 为何 过 不了',
 '为什么 综合 评分 不能 过',
 '请问 下 我 可以 提额',

tfidf-pooling

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
import math 

# 对分词计算tf-idf，取出idf
s= []
for sentence in sentences:
    s.append(' '.join(sentence))
    
tfidf_vec = TfidfVectorizer()
tf_idf = tfidf_vec.fit_transform(s) # 返回样本、特征矩阵

# 输出idf字典
word_idf_dict ={k:v for k,v in zip(tfidf_vec.get_feature_names(),tfidf_vec.idf_)}
word_idf_dict.get('位置')

10.167183477772047

In [58]:
word_tfidf_dict

{'00': 0.0,
 '001234': 0.0,
 '005': 0.0,
 '01': 0.0,
 '0114': 0.0,
 '012': 0.0,
 '021': 0.0,
 '0216': 0.0,
 '03': 0.0,
 '0303': 0.0,
 '035': 0.0,
 '04': 0.0,
 '0471': 0.0,
 '05': 0.0,
 '050': 0.0,
 '051': 0.0,
 '057165111071': 0.0,
 '0602': 0.0,
 '0707': 0.0,
 '09': 0.0,
 '0910': 0.0,
 '0k': 0.0,
 '0pp': 0.0,
 '10': 0.0,
 '100': 0.0,
 '1000': 0.0,
 '10000': 0.0,
 '100000': 0.0,
 '100001': 0.0,
 '1003': 0.0,
 '10050': 0.0,
 '10070': 0.0,
 '103000': 0.0,
 '103616': 0.0,
 '104207': 0.0,
 '1085': 0.0,
 '10w': 0.0,
 '11': 0.0,
 '1100': 0.0,
 '11000': 0.0,
 '1100020': 0.0,
 '110404': 0.0,
 '1125': 0.0,
 '1126': 0.0,
 '113': 0.0,
 '1138': 0.0,
 '115': 0.0,
 '1150': 0.0,
 '1160': 0.0,
 '118': 0.0,
 '12': 0.0,
 '120': 0.0,
 '12000': 0.0,
 '1277': 0.0,
 '13': 0.0,
 '1300': 0.0,
 '13000': 0.0,
 '139': 0.0,
 '14': 0.0,
 '1400': 0.0,
 '14000': 0.0,
 '15': 0.0,
 '150': 0.0,
 '1500': 0.0,
 '15000': 0.0,
 '152': 0.0,
 '1550': 0.0,
 '158': 0.0,
 '16': 0.0,
 '1600': 0.0,
 '16000': 0.0,
 '163': 0.0,
 '16

In [ ]:
# 对每个句子的所有词向量取加权(tfidf)均值，生成句子vector

def build_sentence_weight_vector(sentence,w2v_model,word_tfidf_dict):
    # key_words_list=list(key_weight)
    arr = []
    for word in sentence:
        try:
            if word in word_tfidf_dict.keys():
                arr.append(np.dot(w2v_model.wv[word],math.exp(word_tfidf_dict.get(word))))
            else:
                arr.append(w2v_model.wv[word].reshape(1,100))
        except KeyError:
            continue
        
        return np.vstack(arr).mean(0) 


train['q1_sen_tfidf_vec'] = train['q1_cut'].map(lambda sentence:build_sentence_weight_vector(sentence,w2v_model,word_tfidf_dict))
train['q2_sen_tfidf_vec'] = train['q2_cut'].map(lambda sentence:build_sentence_weight_vector(sentence,w2v_model,word_tfidf_dict))

## 任务5：搭建SiamCNN/LSTM模型，训练和预测

- 步骤1 ：将训练好的word2vex作为深度学习embeeding层的初始化参数。
- 步骤2 ：搭建SiamCNN（Word2Vec句子编码 + 1D CNN +FC）的孪生网络结构，完成训练和预测，提交测试集预测结果。
- 步骤3 ：搭建SiamLSTM（Word2Vec句子编码 + LSTM + FC）的孪生网络结构，完成训练和预测，提交测试集预测结果。


## 任务6：搭建InferSent模型，训练和预测

- 步骤1 ：将训练好的word2vex作为深度学习embeeding层的初始化参数。
- 步骤2 ：搭建InferSent模型，尝试不同的交叉方法。
- 步骤3 ：训练InferSent模型，提交测试集预测结果。

## 任务7：搭建ESIM模型，训练和预测

- 步骤1 ：将训练好的word2vex作为深度学习embeeding层的初始化参数。
- 步骤2 ：搭建ESIM模型，尝试不同的交叉方法。
- 步骤3 ：训练ESIM模型，提交测试集预测结果。


## 任务8：使用BERT或ERNIE完成NSP任务

参考代码：
https://aistudio.baidu.com/aistudio/projectdetail/3168859
bert-nsp代码
- 步骤1 ：学习Bert模型的使用。
- 步骤2 ：使用Bert完成NSP任务的训练和预测，提交测试集预测结果。

## 任务8：Bert-flow、Bert-white、SimCSE（可选，不参与积分）

- 步骤1 ：学习Bert-white原理和实现
- 步骤2 ：学习SimCSE原理和实现